In [33]:
# Copyright 2022 The Forta Foundation
 
import requests
import pandas as pd
import json
import dotenv
import os

# Load environment variables
dotenv.load_dotenv()

True

In [34]:
def execute_query(sql: str) -> pd.DataFrame:
        url = "https://q.luabase.com/run"
        payload = {"block":
                   {"details": {
                    "sql": sql,
                    }
                    },
                   "api_key": os.environ.get('LUABASE_API_KEY')
                   }

        headers = {"content-type": "application/json"}
        response = requests.request("POST", url, json=payload, headers=headers, timeout=600)  # 10min timeout
        
        data = response.json()
        
        print(data)
        return pd.DataFrame(data["data"])

This notebook allows to backtest the attack detector V2 against a set of alerts. Independent of the threshold set within the bot, the log will be analyzed, so the relevant alerts, their anomaly score, overall anomaly score, and any FP filtering mitigations are shown.

Since there is a command line character limit, the local persistence mechanism needs to be enabled to persist state across executions (by setting local node property in constants.py to 1)

In [35]:
start_date = "2022-12-01" #when the attack detector first started to operate

# attacker_addresses = "0xf3a465c9fa6663ff50794c698f600faa4b05c777,0x2ffc59d32a524611bb891cab759112a51f9e33c0" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "ethereum" #ethereum or binance 
# name = "ANKR"
# end_date = "2022-12-04" #should be the date of the attack

start_date = "2023-01-10" #when the attack detector first started to operate
attacker_addresses = "0x9d0163e76bbcf776001e639d65f573949a53ab03"
chain = "ethereum" #ethereum or binance 
name = "LendHub"
end_date = "2023-01-14" #should be the date of the attack

# attacker_addresses = "0x0348d20b74ddc0ac9bfc3626e06d30bb6fac213b"
# chain = "ethereum" #ethereum or binance 
# name = "JayExploiter"
# end_date = "2022-12-31" #should be the date of the attack

# attacker_addresses = "0x001B91c794dFEecf00124D3F9525DD32870B6ee9" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "ethereum" #ethereum or binance 
# name = "CryptoRubric"
# end_date = "2022-12-28" #should be the date of the attack

# attacker_addresses = "0x3bdF01ed32F07e8e843163b5d478d4502F5743CD,0x25fDe76A52D01c83E31d2d3D5e1d2011ff103c56,0xBeAdeDBABED6A353c9cAa4894Aa7E5F883e32967" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "ethereum" #ethereum or binance 
# name = "ElasticSwap"
# start_date = "2022-11-30" #when the attack detector first started to operate
# end_date = "2022-12-15" #should be the date of the attack

# attacker_addresses = "0x86Aa1c46f2Ae35ba1B228dC69fB726813D95b597,0x3c4e5b099f3c02122079d124138377e1b9048629" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "Nimbus"
# end_date = "2022-12-15" #should be the date of the attack

# attacker_addresses = "0xF4FD2EbE7196c8E99E88bcc4Aef69dda0e493B8F" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "BlackGold"
# end_date = "2022-12-15" #should be the date of the attack

# attacker_addresses = "0x286E09932B8D096cbA3423d12965042736b8F850" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "aespool"
# end_date = "2022-12-17" #should be the date of the attack

# attacker_addresses = "0xf71708C59be7e32B9FF2aa174F07311869C6bf0c" #comma separated list of attacker addresses (note, while all alerts from those addresses are pushed to the attack detector, one should check the clustering bot to see whether they are clustered together)
# chain = "binance" #ethereum or binance 
# name = "optionroom"
# end_date = "2022-12-17" #should be the date of the attack




In [36]:
#clustering
for attacker_address in attacker_addresses.split(","):
    sql = f"SELECT metadata FROM forta.{chain}_alerts WHERE bot_id = '0xd3061db4662d5b3406b52b20f34234e462d2c275b99414d76dc644e2486be3e9' and CAST(substring(block_timestamp,1,19) as datetime)  >= '2022-10-01T00:00:00' AND CAST(substring(block_timestamp,1,19)  as datetime)  <= '{end_date}T00:00:00' and  lower(arrayStringConcat(metadata)) like '%{attacker_address.lower()}%'"
    print(sql)
    df = execute_query(sql)
    print(df)

SELECT metadata FROM forta.ethereum_alerts WHERE bot_id = '0xd3061db4662d5b3406b52b20f34234e462d2c275b99414d76dc644e2486be3e9' and CAST(substring(block_timestamp,1,19) as datetime)  >= '2022-10-01T00:00:00' AND CAST(substring(block_timestamp,1,19)  as datetime)  <= '2023-01-14T00:00:00' and  lower(arrayStringConcat(metadata)) like '%0x9d0163e76bbcf776001e639d65f573949a53ab03%'
{'data': [], 'ok': True, 'error': None, 'hit_max_row': False, 'rows': 0, 'meta': [{'name': 'metadata', 'type': 'Array(Map(String, Nullable(String)))'}], 'parameters': {}, 'start_time': '2023-01-14T00:38:19.964495+00:00', 'statistics': {'applied_limit': False, 'blocks': 0, 'bytes': 56136543104, 'calculated_rows_before_limit': 1, 'elapsed': 12.658, 'rows_before_limit': 0, 'rows': 355990024, 'total_rows': 355990024, 'written_bytes': 0, 'written_rows': 0}, 'time_elapsed': 12.678, 'trace': 'projects/luabase/traces/ffb3d585041773f421e96c4cddeb62cd'}
Empty DataFrame
Columns: []
Index: []


In [37]:
alert_hashes_df = pd.DataFrame(columns=["hash","createdAt"])
for attacker_address in attacker_addresses.split(","):
    url = 'https://api.forta.network/graphql'
    query = '''query exampleQuery {
        # first 5 alerts
        alerts(
            input: { first: 200 , addresses: ["'''+attacker_address+'''"],
            blockDateRange: { startDate: "'''+start_date+'''", endDate: "'''+end_date+'''" }
            }
        ) {
            pageInfo {
            hasNextPage
            endCursor {
                alertId
                blockNumber
            }
            }
            alerts {
            hash
            createdAt
            
            }
        }
        }
    '''
    r = requests.post(url, json={'query': query})
    print(r.text)
    json_data = json.loads(r.text)
    df = pd.DataFrame(json_data['data']['alerts']['alerts'])
    print(df)
    alert_hashes_df = pd.concat([alert_hashes_df, df])

len(alert_hashes_df)

{"data":{"alerts":{"pageInfo":{"hasNextPage":false,"endCursor":{"alertId":"","blockNumber":0}},"alerts":[{"hash":"0xd5ac9a876dba63f6d3dc5a3ea85595bad9d7f072f762e75946aaf496370456a1","createdAt":"2023-01-13T08:36:43.935917199Z"},{"hash":"0xad03fb24ac3dab11fb65c68b38680e56dc9e8ff226b21c7550e08405dfcdff70","createdAt":"2023-01-13T08:36:43.94895351Z"},{"hash":"0x85b530b7dbadb55f2688a5497dd2a18c24d054f258ad433eec2c3f576958e212","createdAt":"2023-01-12T19:14:07.23361006Z"},{"hash":"0xc7fcdf9cbb0f1275c2b803874249b5fde5628ffd09a9745a9c217d8ee8f82d5c","createdAt":"2023-01-11T22:57:04.954497958Z"},{"hash":"0xbeb1325ca6f3ffac015fd745c332625784ac9b09846c7392724c6b4d61ab0961","createdAt":"2023-01-13T23:34:35.118811326Z"},{"hash":"0x7fef6d98ac7faf14a4399ec5c13dec18f579c38dc98abe5e1a6828f7146b4ff6","createdAt":"2023-01-13T23:34:33.202879577Z"},{"hash":"0xf88d53335be6082410f3bdd64b9e4e2af3b4b69de2c9f6b247145d9e49453c2f","createdAt":"2023-01-13T11:13:34.504823955Z"},{"hash":"0xc0784f1d3748ddc7f6edafa2e

151

In [38]:
alert_hashes_df

,hash,createdAt
0,0xd5ac9a876dba63f6d3dc5a3ea85595bad9d7f072f762...,2023-01-13T08:36:43.935917199Z
1,0xad03fb24ac3dab11fb65c68b38680e56dc9e8ff226b2...,2023-01-13T08:36:43.94895351Z
2,0x85b530b7dbadb55f2688a5497dd2a18c24d054f258ad...,2023-01-12T19:14:07.23361006Z
3,0xc7fcdf9cbb0f1275c2b803874249b5fde5628ffd09a9...,2023-01-11T22:57:04.954497958Z
4,0xbeb1325ca6f3ffac015fd745c332625784ac9b09846c...,2023-01-13T23:34:35.118811326Z
...,...,...
146,0xe18f80221e23b0b0ec1df67ac3bc060c406e297b54ff...,2023-01-11T22:20:15.592271608Z
147,0xe1e39c3ee33c86354a4731a851f232abe6860c6cf14e...,2023-01-11T22:11:56.789135397Z
148,0xd3291135a14e7e4492be7447874ca40758a1c19f464a...,2023-01-11T22:20:06.199511427Z
149,0xa88eb3f1c4b3e4dd755cb4a8491f3f57aaeda04d0d5c...,2023-01-11T22:27:26.937570743Z


In [45]:
#remove all _key files (L2 cache)
!echo 'output.txt' > {name}.txt

#run npm run to get and route to output.txt
count = 0
alert_hashes = ""
for alert_hash in alert_hashes_df["hash"]:
    count += 1
    alert_hashes += f",{alert_hash}"
    if count % 90 == 0:
        !npm run alert {alert_hashes} >> {name}.txt 2>> {name}.txt
        alert_hashes = ""
if count % 90 != 0:
    !npm run alert {alert_hashes} >> {name}.txt 2>> {name}.txt



In [46]:
!grep 'ERROR' {name}.txt


ERROR: RangeError: Invalid string length
2023-01-13 17:10:53,416 - root - ERROR - Failed to populate denominator cache for 1 transfer-in 2023-01-10 14:29:23 2023-01-11 14:29:23: HTTPSConnectionPool(host='q.luabase.com', port=443): Read timed out.
ERROR: RangeError: Invalid string length


In [47]:
!grep 'WARN' {name}.txt

2023-01-13 17:11:07,003 - root - WARNING - Exception in get_denominator Denominator not found for 1 transfer-in 2023-01-10 14:29:23 2023-01-11 14:29:23 in cache


In [48]:
!grep 'adding cluster mapping' {name}.txt

In [49]:
!grep 'adding FP mitigation cluster' {name}.txt

2023-01-13 17:12:17,015 - root - INFO - alert 0xa88eb3f1c4b3e4dd755cb4a8491f3f57aaeda04d0d5cab53cfa06d90ae48f580 adding FP mitigation cluster: 0x9D0163e76BbCf776001E639d65F573949a53AB03. FP mitigation clusters size now: 0


In [50]:
#read output.txt and answer questions around
!grep 'anomaly' {name}.txt


#anomaly scores
#FP filtering


2023-01-13 16:44:27,397 - root - INFO - alert 0x7fef6d98ac7faf14a4399ec5c13dec18f579c38dc98abe5e1a6828f7146b4ff6 0x4adff9a0ed29396d51ef3b16297070347aab25575f04a4e2bd62ec43ca4508d2 POSSIBLE-MONEY-LAUNDERING-TORNADO-CASH MoneyLaundering: 0x9d0163e76bbcf776001e639d65f573949a53ab03 anomaly score of 0.0008396305625524769
2023-01-13 16:45:11,843 - root - INFO - alert 0xfaeb83c919c1232d98108a126e8d4c34844af7ff617e1c4055e6f882031801ff 0x4adff9a0ed29396d51ef3b16297070347aab25575f04a4e2bd62ec43ca4508d2 POSSIBLE-MONEY-LAUNDERING-TORNADO-CASH MoneyLaundering: 0x9d0163e76bbcf776001e639d65f573949a53ab03 anomaly score of 0.0007062146892655367
2023-01-13 16:45:19,344 - root - INFO - alert 0xbe0d5e5022179482a4bcfbcf46972982c3d4481143dd8d80b44b77fc4660a7e7 0xaf9ac4c204eabdd39e9b00f91c8383dc01ef1783e010763cad05cc39e82643bb LARGE-TRANSFER-OUT MoneyLaundering: 0x9d0163e76bbcf776001e639d65f573949a53ab03 anomaly score of 0.0007062146892655367
2023-01-13 16:45:28,447 - root - INFO - alert 0x7b8f5009eb8a73e464